# Multivariate Linear Regression
Multivariate Linear Regression with Backward Elimination to have features that are significant to the model
### Step 1: Importing required libraries and modules

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from statsmodels.formula.api import OLS as regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

### Step 2: Reading CSV dataset to pandas Dataframe

In [22]:
dframe = pd.read_csv("50_startups.csv")
dframe.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


Copying X training data to a new dataframe

In [23]:
x_data_frame = dframe[dframe.columns.tolist()[:-1]]
x_data_frame.head()

,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


Similarly, copying Y data to a new dataframe

In [24]:
y_data_frame = dframe.Profit
y_data_frame.head()

0    192261.83
1    191792.06
2    191050.39
3    182901.99
4    166187.94
Name: Profit, dtype: float64

### Step 3: Label Encoding
To convert categorical data(State) to numerical data for model fitting and prediction

In [25]:
label_encoder = LabelEncoder()
x_data_frame.State = label_encoder.fit_transform(x_data_frame.State)
x_data_frame.head()

C:\IntelPython3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,2
1,162597.70,151377.59,443898.53,0
2,153441.51,101145.55,407934.54,1
3,144372.41,118671.85,383199.62,2
4,142107.34,91391.77,366168.42,1


> To avoid multi colinearity, eliminate the relation between the encoded labels, use OneHotEncoding on encoded labels<br>

**In this example, i got rid of the Dummy Variable Trap by eliminating very first column of training data.**

In [26]:
hot_encoder = OneHotEncoder(categorical_features=[3])
x_data = hot_encoder.fit_transform(x_data_frame).toarray()[:,1:]
x_data = np.append(np.ones((x_data.shape[0], 1)), x_data, axis=1).astype(np.int)
x_data[:5,:]

array([[     1,      0,      1, 165349, 136897, 471784],
       [     1,      0,      0, 162597, 151377, 443898],
       [     1,      1,      0, 153441, 101145, 407934],
       [     1,      0,      1, 144372, 118671, 383199],
       [     1,      1,      0, 142107,  91391, 366168]])

In [27]:
y_data = y_data_frame.values.astype(np.int)
y_data[:5]

array([192261, 191792, 191050, 182901, 166187])

### Step 4: Split and Fit
In this step, i am spitting the data to test and training set for x and y repectively and then fitting the training data to the regression model. 
<br>
In this case, we are not using scikit Regression Models because they do not provide P value information for the fitted model and thus i am making use of statsmodels in python for Regression Model

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=0)
regressor = regression(exog=x_train, endog=y_train).fit()
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     129.7
Date:                Fri, 08 Sep 2017   Prob (F-statistic):           3.91e-21
Time:                        08:11:37   Log-Likelihood:                -421.10
No. Observations:                  40   AIC:                             854.2
Df Residuals:                      34   BIC:                             864.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.255e+04   8358.564      5.091      0.000    2.56e+04    5.95e+04
x1          -959.4172   4038.118     -0.238      0.814   -9165.859    7247.025
x2           699.3138   3661.573      0.191      0.850   -6741.898    8140.526
x3             0.7735      0.055     14.025      0.000       0.661       0.886
x4             0.0329      0.066      0.495      0.624      -0.102       0.168
x5             0.0366      0.019      1.884      0.068      -0.003       0.076
==============================================================================
Omnibus:                       15.823   Durbin-Watson:                   2.468
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.232
Skew:                          -1.094   Prob(JB):                     9.02e-06
Kurtosis:                       6.026   Cond. No.                     1.49e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Step 5: Building a better model with Backward Elimination
For model for better prediction, i am making use of Backward elimination process to get rid of the features that are not really affecting the performance of the model.

Backward Elimination steps:
- Suppose a significance value, let say 5%
- Fit the model with all features and check for a feature having P value > Significance value, otherwise finish
- Eliminate that feature having P value > Significance value and fit the model with new feature set
- Repeat until we get rid of all the unwanted features from the training set

In this case, we get rid of x2 (refer above summary) from training set having P value of85% (0.850) and then fit the model with remaining features and check for the feature having P value > Significance value again.

In [29]:
x_train = x_train[:,[0,2,3,4,5]]
x_test = x_test[:,[0,2,3,4,5]]
regressor = regression(exog=x_train, endog=y_train).fit()
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     166.6
Date:                Fri, 08 Sep 2017   Prob (F-statistic):           2.90e-22
Time:                        08:11:54   Log-Likelihood:                -421.13
No. Observations:                  40   AIC:                             852.3
Df Residuals:                      35   BIC:                             860.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.241e+04   8223.460      5.157      0.000    2.57e+04    5.91e+04
x1          1052.2133   3301.357      0.319      0.752   -5649.897    7754.324
x2             0.7747      0.054     14.302      0.000       0.665       0.885
x3             0.0318      0.065      0.487      0.630      -0.101       0.165
x4             0.0357      0.019      1.899      0.066      -0.002       0.074
==============================================================================
Omnibus:                       15.352   Durbin-Watson:                   2.477
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.376
Skew:                          -1.093   Prob(JB):                     2.28e-05
Kurtosis:                       5.836   Cond. No.                     1.48e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.48e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now, as we see the P value for x1 feature is way higher then significance value, then we get rid of this feature and fit the model again

In [30]:
x_train = x_train[:,[0,2,3,4]]
x_test = x_test[:,[0,2,3,4]]
regressor = regression(exog=x_train, endog=y_train).fit()
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     227.8
Date:                Fri, 08 Sep 2017   Prob (F-statistic):           1.85e-23
Time:                        08:14:08   Log-Likelihood:                -421.19
No. Observations:                  40   AIC:                             850.4
Df Residuals:                      36   BIC:                             857.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.299e+04   7919.796      5.428      0.000    2.69e+04    5.91e+04
x1             0.7788      0.052     15.003      0.000       0.674       0.884
x2             0.0294      0.064      0.458      0.650      -0.101       0.160
x3             0.0347      0.018      1.896      0.066      -0.002       0.072
==============================================================================
Omnibus:                       15.557   Durbin-Watson:                   2.481
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.540
Skew:                          -1.081   Prob(JB):                     1.27e-05
Kurtosis:                       5.974   Cond. No.                     1.43e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now, x2 feature has P value greater then 5%, so we will eliminate this in the following step.

In [31]:
x_train = x_train[:,[0,1,3]]
x_test = x_test[:,[0,1,3]]
regressor = regression(exog=x_train, endog=y_train).fit()
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                     349.0
Date:                Fri, 08 Sep 2017   Prob (F-statistic):           9.65e-25
Time:                        08:16:25   Log-Likelihood:                -421.30
No. Observations:                  40   AIC:                             848.6
Df Residuals:                      37   BIC:                             853.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.635e+04   2971.236     15.598      0.000    4.03e+04    5.24e+04
x1             0.7886      0.047     16.846      0.000       0.694       0.883
x2             0.0326      0.018      1.860      0.071      -0.003       0.068
==============================================================================
Omnibus:                       14.666   Durbin-Watson:                   2.518
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               20.583
Skew:                          -1.030   Prob(JB):                     3.39e-05
Kurtosis:                       5.847   Cond. No.                     4.97e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

> Here is very interesting point which i would like to describe and is about feature x2 having P value of 7.1% which is still greater than signficance value. But, i would not opt out from this feature to prevent meaningful data to the model.

Now, i didn't find any feature to be eliminated. Thus, by now, our model is ready for prediction.

Final Adjusted R<sup>2</sup> is 94.7%